<a href="https://colab.research.google.com/github/VineetMakharia/NN-ipynbs/blob/main/Copy_of_NN_IOT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
 from google.colab import files 
  
  
uploaded = files.upload()

Saving 9.csv to 9.csv


In [ ]:
import pandas as pd 
import io 
  
df = pd.read_csv(io.BytesIO(uploaded['9.csv']), names= ['x1', 'x2', 'x3', 'x4', 'x5', 'y']) 

In [ ]:
X = df[['x1','x2','x3','x4','x5']]
X.head()

,x1,x2,x3,x4,x5
0,43.060,80.829,154.37,205.63,253.82
1,54.100,79.580,118.67,207.98,257.62
2,49.086,76.696,136.36,212.10,243.83
3,35.767,91.612,122.82,236.30,245.87
4,55.154,85.700,116.98,245.92,262.81


In [ ]:
Y = df[['y']]
Y.head()

,y
0,5659.4
1,5517.6
2,5514.7
3,5662.9
4,5864.4


In [ ]:
from sklearn.model_selection import train_test_split
# Total 2300 samples, need 300 for test, so testsize = 300/2300 ~= 0.13
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.13)
assert len(X_train) + len(X_test) == len(X)

In [ ]:
# Need to use a scaler for a neural network, popular ones are Standard Scaler and Min Max Scaler
# It is important to scale our train and test separately else we might get skewed results.
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(X_train)

X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

In [ ]:
X_train.shape

(2001, 5)

In [ ]:
y_train.shape

(2001, 1)

In [ ]:
X_test.shape

(299, 5)

In [ ]:
y_test.shape

(299, 1)

In [ ]:
X.shape

(2300, 5)

In [ ]:
Y.shape

(2300, 1)

In [ ]:
from sklearn.model_selection import GridSearchCV, KFold
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasRegressor
from keras.optimizers import Adam
from keras.layers import Dropout

### Creating a base model with some parameters to find the R2 value, we are looking to get closer to a value of 1. 

In [ ]:
def base_model(unit = 256,kernel_initializer='normal',optimizer='adam'):
  model = Sequential()
  model.add(Dense(128, kernel_initializer='normal',input_dim = 5, activation='relu'))

  # Creating 3 hidden layers
  model.add(Dense(units=unit, kernel_initializer=kernel_initializer,activation='relu'))
  model.add(Dense(units=unit, kernel_initializer=kernel_initializer,activation='relu'))
  model.add(Dense(units=unit, kernel_initializer=kernel_initializer,activation='relu'))

  # The Output Layer :
  model.add(Dense(1, kernel_initializer=kernel_initializer,activation='linear'))

  model.compile(loss='mean_squared_error', optimizer=optimizer)
  model.summary()
  return model

In [ ]:
model = KerasRegressor(build_fn=base_model)
model.fit(X_train, y_train,
          epochs=100)
y_pred = model.predict(X_test)

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_20 (Dense)             (None, 128)               768       
_________________________________________________________________
dense_21 (Dense)             (None, 256)               33024     
_________________________________________________________________
dense_22 (Dense)             (None, 256)               65792     
_________________________________________________________________
dense_23 (Dense)             (None, 256)               65792     
_________________________________________________________________
dense_24 (Dense)             (None, 1)                 257       
Total params: 165,633
Trainable params: 165,633
Non-trainable params: 0
_________________________________________________________________
Epoch 1/100
63/63 [==============================] - 0s 3ms/step - loss: 25042490.0000
Epoch 2/100
63/63 [========

In [ ]:
from sklearn.metrics import r2_score
r2_score(y_test,y_pred)

0.9968433075389292

# Now with our base model, we are getting close to 1, we can do the hyper   parameter tuning

In [ ]:
# Need a Keras wrapper over the base_model 
classifier = KerasRegressor(build_fn=base_model)

In [ ]:
from sklearn.model_selection import GridSearchCV
# parameters ={'nb_epoch':[100,250,500,1000,2000],
#             'kernel_initializer':['random_uniform', 'normal'],
#             'unit':[128,256,512],
#              'optimizer':['adam','rmsprop','SGD']
#              }
parameters ={'nb_epoch':[100,250,500,1000,2000],
            'kernel_initializer':['random_uniform', 'normal'],
            'unit':[128,256,512],
             }
gridSearch = GridSearchCV(classifier,param_grid=parameters,
                          cv=4,
                         n_jobs=-1,
                         return_train_score=True,
                          scoring='r2',
                          verbose = 2
                          )

In [ ]:
import tensorflow as tf
tf.keras.backend.set_floatx('float64')
hist = gridSearch.fit(X_train, y_train)

Fitting 4 folds for each of 30 candidates, totalling 120 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  37 tasks      | elapsed:   29.1s
[Parallel(n_jobs=-1)]: Done 120 out of 120 | elapsed:  1.4min finished


Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_25 (Dense)             (None, 128)               768       
_________________________________________________________________
dense_26 (Dense)             (None, 512)               66048     
_________________________________________________________________
dense_27 (Dense)             (None, 512)               262656    
_________________________________________________________________
dense_28 (Dense)             (None, 512)               262656    
_________________________________________________________________
dense_29 (Dense)             (None, 1)                 513       
Total params: 592,641
Trainable params: 592,641
Non-trainable params: 0
_________________________________________________________________
63/63 [==============================] - 1s 14ms/step - loss: 18097920.3179


In [ ]:
print('Grid Search Best score',hist.best_score_)
print('Grid Search Best Parameters', hist.best_params_)
print('Execution time',hist.refit_time_)

Grid Search Best score -42.36463940500185
Grid Search Best Parameters {'kernel_initializer': 'random_uniform', 'nb_epoch': 500, 'unit': 512}
Execution time 1.5915100574493408


In [ ]:
model1 = KerasRegressor(build_fn = base_model,verbose = 0,epochs = 500,kernel_initializer='random_uniform',unit = 512, batch_size = 128)

In [ ]:
model1.fit(X_train,y_train)

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_30 (Dense)             (None, 128)               768       
_________________________________________________________________
dense_31 (Dense)             (None, 512)               66048     
_________________________________________________________________
dense_32 (Dense)             (None, 512)               262656    
_________________________________________________________________
dense_33 (Dense)             (None, 512)               262656    
_________________________________________________________________
dense_34 (Dense)             (None, 1)                 513       
Total params: 592,641
Trainable params: 592,641
Non-trainable params: 0
_________________________________________________________________


In [ ]:
y_pred1 = model1.predict(X_test)
print(y_test.head())
print(y_pred[:5])

           y
35    5255.9
1522  5462.9
841   5558.2
2072  5568.0
1592  5641.8
[5251.773 5456.663 5556.151 5553.633 5626.014]


In [ ]:
from sklearn.metrics import mean_squared_error
test_loss = []
test_loss.append(mean_squared_error(y_test, y_pred1))
print("Test loss: {}".format(test_loss))

Test loss: [1073.24920015419]


In [ ]:
from sklearn.metrics import r2_score
print(r2_score(y_test,y_pred1))

0.9792278996039719


In [ ]:
import statsmodels.api as sm
import numpy as np
def LR(Xc,Yc):
    Xc = sm.add_constant(Xc)
    model = sm.OLS(Yc,Xc).fit()
    print(model.params)
    print(model.summary())
    return model

In [ ]:
y_predict = LR(X,Y)

const    629.977241
x1         2.531700
x2         3.892072
x3         5.986373
x4         7.554291
x5         8.404686
dtype: float64
                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.998
Model:                            OLS   Adj. R-squared:                  0.998
Method:                 Least Squares   F-statistic:                 2.768e+05
Date:                Wed, 04 Nov 2020   Prob (F-statistic):               0.00
Time:                        22:05:12   Log-Likelihood:                -8318.3
No. Observations:                2300   AIC:                         1.665e+04
Df Residuals:                    2294   BIC:                         1.668e+04
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.9

In [ ]:
lr_save = y_predict.predict(X_test)

ValueError: ignored

In [ ]:
from sklearn.metrics import mean_squared_error

In [ ]:
test_loss = []
test_loss.append(mean_squared_error(y_test, lr_save))

In [ ]:
print(test_loss)

[30882917.89119411]


In [ ]:
lr_save.shape

(299,)

In [ ]:
error = np.subtract(y_test,lr_save)

ValueError: ignored